# Adoption

In [ ]:
import numpy as np
import sqlite3 as sq
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import Counter
import requests
import time
import json
import importlib
import tldextract
from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
def toText(row):
    if row["q_version"] == 4278190109:
        return "Draft 29"
    elif row["q_version"] == 4278190112:
        return "Draft 32"
    elif row["q_version"] == 4278190114:
        return "Draft 34"
    elif row["q_version"] == 1:
        return "1"
    else:
        return "Unknown"

def qVersionToText(row):
    if row["q_version"] == 4278190109:
        return "QUIC_Draft_29"
    elif row["q_version"] == 4278190112:
        return "QUIC_Draft_32"
    elif row["q_version"] == 4278190114:
        return "QUIC_Draft_34"
    elif row["q_version"] == 1:
        return "QUIC_1"
    else:
        return "QUIC_Unknown"

def doqVersionToText(row):
    if row["doq_version"] == "doq-i00":
        return "DoQ_Draft_00"
    if row["doq_version"] == "doq-i01":
        return "DoQ_Draft_01"
    if row["doq_version"] == "doq-i02":
        return "DoQ_Draft_02"
    if row["doq_version"] == "doq-i03":
        return "DoQ_Draft_03"
    if row["doq_version"] == "doq-i04":
        return "DoQ_Draft_04"
    if row["doq_version"] == "doq-i05":
        return "DoQ_Draft_05"
    if row["doq_version"] == "doq-i06":
        return "DoQ_Draft_06"
    else:
        return "DoQ_Unknown"
    
def qodQuicVersionPivot(quic_doq_versions):
    quic_doq_versions_grouped = quic_doq_versions.groupby(['WeekNumber', 'doq_quic'])['ip'].size().reset_index(name = "total_per_week")

    quic_doq_versions_grouped_pivot = quic_doq_versions_grouped.pivot(index = "WeekNumber", columns = "doq_quic", values = "total_per_week")
    quic_doq_versions_grouped_pivot = quic_doq_versions_grouped_pivot.astype(pd.Int64Dtype())
    quic_doq_versions_grouped_pivot = quic_doq_versions_grouped_pivot.fillna(0)
    
    return quic_doq_versions_grouped_pivot
        
def buildExtractedDomain(extracted):
    result = ""
    if extracted.subdomain != "" and extracted.subdomain != "*":
        result += extracted.subdomain + "."
    if extracted.domain != "" and extracted.suffix != "":
        result += extracted.domain + "." + extracted.suffix
    return result
    
def mapToCommonName(info):
    commonName = buildExtractedDomain(tldextract.extract(info["CommonName"]))
    if commonName != "":
        return commonName
    if info["DNSNames"] == None:
        return None
    return buildExtractedDomain(tldextract.extract(info["DNSNames"][0]))


In [ ]:
connection_complete = sq.connect("./misc/merged-complete.db")
connection_complete.row_factory = sq.Row

connection_q_versions_cleaned = sq.connect("./misc/merged-q_versions_cleaned.db")
connection_q_versions_cleaned.row_factory = sq.Row

# ---------------------------------------------------------------------------------

## Remove duplicates due to targets offering DoQ on multiple ports

In [ ]:
sql_misc_df = """
select 
    strftime('%W', created_at) week_number, *
from q_versions
order by week_number asc;
"""

misc_df = pd.read_sql_query(sql_misc_df, connection_complete)

In [ ]:
misc_df = misc_df.drop_duplicates(subset = ['week_number', 'ip', 'q_version', 'draft_version'])

In [ ]:
conn = sq.connect('./misc/merged-q_versions_cleaned.db')
misc_df.to_sql('q_versions_cleaned', conn, if_exists='replace', index=False)

# ---------------------------------------------------------------------------------

## QUIC versions over Time

In [ ]:
sqlQuicVersions = """
select 
    strftime('%Y-%W', created_at) WeekNumber,
    q_version, count(*) as count
from q_versions_cleaned
group by WeekNumber, q_version
order by count(*) desc;
"""

quic_versions = pd.read_sql_query(sqlQuicVersions, connection_q_versions_cleaned)

quic_versions["version_name"] = quic_versions.apply(toText, axis = 1)
quic_versions_per_week_total = quic_versions.groupby(["WeekNumber"])["count"].sum().reset_index(name = "total")
quic_versions = pd.merge(quic_versions, quic_versions_per_week_total, on = "WeekNumber")
quic_versions["percentage"] = (quic_versions["count"] / quic_versions["total"]) * 100

In [ ]:
quic_versions_by_draft = quic_versions.pivot(index = "WeekNumber", columns = "version_name", values = "count")
quic_versions_by_draft = quic_versions_by_draft.fillna(0)

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = fig.add_subplot(1,1,1)

groups = [quic_versions_by_draft[group].tolist() for group in quic_versions_by_draft]

labels = [i for i in range(27, len(quic_versions_by_draft.index) + 27)]

plt.plot(labels, quic_versions_by_draft["1"], color="tab:grey", label="1")
plt.plot(labels, quic_versions_by_draft["Draft 29"], color="tab:blue", label="Draft 29")
plt.plot(labels, quic_versions_by_draft["Draft 32"], color="tab:green", label="Draft 32",)
plt.plot(labels, quic_versions_by_draft["Draft 34"], color="tab:orange", label="Draft 34")

plt.xlabel("Week Number")
plt.ylabel("% of Resolvers")
plt.xticks(labels)
ax.set_axisbelow(True)
plt.grid()
plt.yticks([i for i in range(0, 1200, 100)])

plt.legend()
plt.show()

# ---------------------------------------------------------------------------------

## DoQ versions over Time

In [ ]:
sqlQuicVersions = """
select 
    strftime('%Y-%W', created_at) WeekNumber,
    draft_version, count(*) as count
from q_versions_cleaned
group by WeekNumber, draft_version;
"""

draft_versions = pd.read_sql_query(sqlQuicVersions, connection_q_versions_cleaned)

draft_versions_per_week_total = draft_versions.groupby(["WeekNumber"])["count"].sum().reset_index(name = "total")
draft_versions = pd.merge(draft_versions, draft_versions_per_week_total, on = "WeekNumber")
draft_versions["percentage"] = (draft_versions["count"] / draft_versions["total"]) * 100

In [ ]:
draft_versions_by_draft = draft_versions.pivot(index = "WeekNumber", columns = "draft_version", values = "count")

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = fig.add_subplot(1,1,1)

labels = [i for i in range(27, len(draft_versions_by_draft.index) + 27)]

plt.plot(labels, draft_versions_by_draft["doq-i00"], color="tab:blue", label="Draft version 0")
plt.plot(labels, draft_versions_by_draft["doq-i02"], color="tab:green", label="Draft version 2",)
plt.plot(labels, draft_versions_by_draft["doq-i03"], color="tab:gray", label="Draft version 3")

plt.xlabel("Week Number")
plt.ylabel("% of Resolvers")
plt.xticks(labels)
ax.set_axisbelow(True)
plt.grid()
plt.yticks([i for i in range(0, 1200, 100)])

plt.legend()
plt.show()

# ---------------------------------------------------------------------------------

## DoQ + QUIC versions over Time

In [ ]:
sqlQuicVersions = """
select 
    strftime('%Y-%W', created_at) WeekNumber,
    q_version, draft_version as doq_version, ip
from q_versions_cleaned;
"""

quic_doq_versions = pd.read_sql_query(sqlQuicVersions, connection_q_versions_cleaned)

quic_doq_versions["q_version_name"] = quic_doq_versions.apply(qVersionToText, axis = 1)
quic_doq_versions["doq_version_name"] = quic_doq_versions.apply(doqVersionToText, axis = 1)
quic_doq_versions["doq_quic"] = quic_doq_versions["doq_version_name"] + "/" + quic_doq_versions["q_version_name"]

quic_doq_versions_pivot = qodQuicVersionPivot(quic_doq_versions)

quic_doq_versions_pivot.columns = [x.replace('_', ' ') for x in quic_doq_versions_pivot.columns]

In [ ]:
quic_doq_versions_pivot

In [ ]:
from cycler import cycler
custom_cycler = (cycler(color=["#E69F00", "#56B4E9", "#0072B2", "#009E73", "#CC79A7", "#D55E00", "#F0E442"]))  # colors from https://ranocha.de/blog/colors/

tmp_df = quic_doq_versions_pivot.T

tmp_df.columns=['27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
               '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '01', '02', '03']

fig, ax = plt.subplots(figsize=(16,4))
ax.set_prop_cycle(custom_cycler)

patches = []
bottoms = []

for ind in tmp_df.index:
    vals = tmp_df.loc[ind].to_list()
    if bottoms:
        p = ax.bar(tmp_df.columns, vals, label=ind,
                   bottom=bottoms, width=0.7)
    else:
        p = ax.bar(tmp_df.columns, vals, label=ind, width=0.7)
    
    if bottoms:
        bottoms = [b+v for b, v in zip(bottoms, vals)]
    else:
        bottoms = vals
    patches.append(p)

ax.set_xlabel('Week Number of 2021 / 2022')
ax.set_ylabel('Number of DoQ-verified Resolvers')

ax.set_yticks(np.arange(0,1250,100))

handles, labels = ax.get_legend_handles_labels()
#labels = [x.replace('/', ' / \n') for x in labels]  # use newline in legend label to reduce clutter
labels = [labels[0], labels[1], labels[3], labels[4], labels[5], labels[2], labels[6]]
handles = [handles[0], handles[1], handles[3], handles[4], handles[5], handles[2], handles[6]]
ax.legend(handles, labels)

ax.legend(handles, labels, bbox_to_anchor=(0.01,1.17), loc='upper left', labelspacing = 0.8, ncol=4, columnspacing=2, frameon=False)


ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)


# total values
ax.bar_label(patches[-1], tmp_df.sum(axis=0).to_list(), fontweight='bold')

# individual values
for i in range(len(patches)):
    # 'mute' 0 in labels
    ax.bar_label(container=patches[i], labels=[x if x>=100 else '' for x in patches[i].datavalues],
                 label_type='center', fontsize='small')

ax.margins(0.01, 0)
    
plt.show()

# ---------------------------------------------------------------------------------

## Number of Doq verified targets

### Unique over all weeks

In [ ]:
sqlVerifiedTargets = """
select week_number, ip from q_versions_cleaned;
"""

verified_targets = pd.read_sql_query(sqlVerifiedTargets, connection_q_versions_cleaned)

verified_targets['ip'].unique().size

### Verified in first/last week

In [ ]:
verified_targets_first_week = verified_targets[verified_targets['week_number'] == '27']['ip']
print('27: ' + str(len(verified_targets_first_week)))
verified_targets_last_week = verified_targets[verified_targets['week_number'] == '03']['ip']
print('03: ' + str(len(verified_targets_last_week)))
print('increase 27 to 03: ' + str(len(verified_targets_last_week) / len(verified_targets_first_week)))
print('still verified in 03: ' + str(len(set(verified_targets_first_week).intersection(set(verified_targets_last_week)))))
print('still verified in 03 rel: ' + str(len(set(verified_targets_first_week).intersection(set(verified_targets_last_week))) / len(verified_targets_first_week)))


# ---------------------------------------------------------------------------------

## DoUDP

In [ ]:
# first week
doudp_ips_first_week = pd.read_csv('adoption/05-07-2021-53.csv')
len(doudp_ips_first_week)

In [ ]:
# last week
doudp_ips_last_week = pd.read_csv('adoption/17-01-2022-53.csv')
len(doudp_ips_last_week)

In [ ]:
# Still online in the last week
doudp_ips_still_verified_last_week = set(doudp_ips_first_week[doudp_ips_first_week.columns[0]]).intersection(doudp_ips_last_week[doudp_ips_last_week.columns[0]])

In [ ]:
len(doudp_ips_still_verified_last_week) / len(doudp_ips_first_week)

# ---------------------------------------------------------------------------------

## Common Names

### Unique over all weeks

In [ ]:
certs = pd.read_csv("./misc/certs-complete.csv", converters={"info":json.loads}, header = None, names = ["ip", "protocol", "port", "info"])
quic_certs_complete = certs[certs["protocol"] == "quic"]
quic_certs_complete["common_name"] = quic_certs_complete["info"].apply(mapToCommonName)

quic_certs_complete['common_name'].unique().size


### Last week

In [ ]:
certs = pd.read_csv("./misc/certs-last.csv", converters={"info":json.loads}, header = None, names = ["ip", "protocol", "port", "info"])
quic_certs_last = certs[certs["protocol"] == "quic"]
quic_certs_last["common_name"] = quic_certs_last["info"].apply(mapToCommonName)

In [ ]:
quic_certs_last = quic_certs_last.drop_duplicates(subset = ['ip', 'common_name'])

In [ ]:
count_by_common_name = quic_certs_last.groupby("common_name").size().reset_index(name = "count").set_index("common_name")
count_by_common_name["rel"] = (count_by_common_name["count"] / count_by_common_name["count"].sum()) * 100
count_by_common_name.nlargest(10, "count")

### Matching 'adguard'

In [ ]:
quic_certs_last_adguard = certs[certs["protocol"] == "quic"]
quic_certs_last_adguard["common_name"] = quic_certs_last_adguard["info"].apply(mapToCommonName)
quic_certs_last_adguard = quic_certs_last_adguard.drop_duplicates(subset = ['ip', 'common_name'])
quic_certs_last_adguard = quic_certs_last_adguard[['common_name']]

quic_certs_last_adguard = quic_certs_last_adguard[quic_certs_last_adguard['common_name'].str.contains('adguard', case=False, regex=False, na=False)]
quic_certs_last_adguard = quic_certs_last_adguard.drop_duplicates(subset = ['common_name'])
quic_certs_last_adguard = quic_certs_last_adguard.drop(quic_certs_last_adguard[(quic_certs_last_adguard['common_name'] == 'dns.adguard.com') | (quic_certs_last_adguard['common_name'] == '*.d.adguard-dns.com') | (quic_certs_last_adguard['common_name'] == 'adguard.ch')].index)
print(len(quic_certs_last_adguard))
quic_certs_last_adguard

## nextdns.io over time

In [ ]:
quic_certs_nextdns = quic_certs_complete[quic_certs_complete['common_name'] == 'dns.nextdns.io']

In [ ]:
nextdns_ips = quic_certs_nextdns['ip']

In [ ]:
quic_doq_versions_nextdns = quic_doq_versions[quic_doq_versions['ip'].isin(nextdns_ips)]

In [ ]:
quic_doq_versions_pivot_nextdns = qodQuicVersionPivot(quic_doq_versions_nextdns)
quic_doq_versions_pivot_nextdns

In [ ]:
# resolver still with DoQ02/QUIC32 in Weeks 33 and 34
quic_doq_versions_nextdns[((quic_doq_versions_nextdns['WeekNumber'] == '2021-33') | (quic_doq_versions_nextdns['WeekNumber'] == '2021-34')) & (quic_doq_versions_nextdns['q_version_name'] == 'QUIC_Draft_32')]


In [ ]:
# resolver still with DoQ02/QUIC32 in Weeks 33 and 34 changed to DoQ02/QUIC29 in week 35
quic_doq_versions_nextdns_doq02_quic32_weeks_33_34_ip = quic_doq_versions_nextdns[((quic_doq_versions_nextdns['WeekNumber'] == '2021-33') | (quic_doq_versions_nextdns['WeekNumber'] == '2021-34')) & (quic_doq_versions_nextdns['q_version_name'] == 'QUIC_Draft_32')]['ip'][0:1]
quic_doq_versions_nextdns[quic_doq_versions_nextdns['ip'].isin(quic_doq_versions_nextdns_doq02_quic32_weeks_33_34_ip)]


In [ ]:
# mean resolvers in weeks 27-31
(quic_doq_versions_pivot_nextdns[0:5]["DoQ_Draft_02/QUIC_Draft_29"].sum() + quic_doq_versions_pivot_nextdns[0:5]["DoQ_Draft_02/QUIC_Draft_32"].sum()) / 5


In [ ]:
# mean resolvers in weeks 32-03
(quic_doq_versions_pivot_nextdns[5:]["DoQ_Draft_02/QUIC_Draft_29"].sum() + quic_doq_versions_pivot_nextdns[5:]["DoQ_Draft_02/QUIC_Draft_32"].sum() + quic_doq_versions_pivot_nextdns[5:]["DoQ_Draft_02/QUIC_1"].sum()) / 24


In [ ]:
## DoQ02/QUIC29 is only offered by nextdns
quic_doq_versions_doq02_quic_29 = quic_doq_versions[quic_doq_versions['doq_quic'] == 'DoQ_Draft_02/QUIC_Draft_29']
quic_doq_versions_doq02_quic_29[~quic_doq_versions['ip'].isin(nextdns_ips)]

## AdGuard over time

In [ ]:
quic_certs_complete['common_name']

In [ ]:
quic_certs_adguard = quic_certs_complete[(quic_certs_complete['common_name'] == 'dns.adguard.com') | (quic_certs_complete['common_name'] == '*.d.adguard-dns.com') | (quic_certs_complete['common_name'] == 'adguard.ch')]

print(quic_certs_adguard[quic_certs_adguard['common_name'] == 'dns.adguard.com']['info'].iloc[1])
print(quic_certs_adguard[quic_certs_adguard['common_name'] == '*.d.adguard-dns.com']['info'].iloc[1])
print(quic_certs_adguard[quic_certs_adguard['common_name'] == 'adguard.ch']['info'].iloc[1])



In [ ]:
quic_certs_adguard = quic_certs_adguard.drop_duplicates('ip')

In [ ]:
adguard_ips = quic_certs_adguard['ip']

In [ ]:
quic_doq_versions_adguard = quic_doq_versions[quic_doq_versions['ip'].isin(adguard_ips)]

In [ ]:
quic_doq_versions_certs_adguard = pd.merge(quic_doq_versions_adguard, quic_certs_adguard, left_on='ip', right_on='ip', how='left')
quic_doq_versions_certs_adguard_grouped = quic_doq_versions_certs_adguard.groupby(['WeekNumber', 'common_name'])['ip'].size().reset_index(name = "total_per_week")
quic_doq_versions_certs_adguard_grouped_pivot = quic_doq_versions_certs_adguard_grouped.pivot(index = "WeekNumber", columns = "common_name", values = "total_per_week")
quic_doq_versions_certs_adguard_grouped_pivot = quic_doq_versions_certs_adguard_grouped_pivot.astype(pd.Int64Dtype())
quic_doq_versions_certs_adguard_grouped_pivot = quic_doq_versions_certs_adguard_grouped_pivot.fillna(0)
quic_doq_versions_certs_adguard_grouped_pivot


In [ ]:
quic_doq_versions_pivot_adguard = qodQuicVersionPivot(quic_doq_versions_adguard)
quic_doq_versions_pivot_adguard

# ---------------------------------------------------------------------------------

# Regional Distribution last week

In [ ]:
# get regional data from ip addresses
data = {}
try:
    df = pd.read_csv("adoption-ip-location.csv")
    for index, row in df.iterrows():
        data[row["ip"]] = row
except:
    pass
for row in verified_targets_last_week:
    #print(row)
    r = requests.get("http://ip-api.com/json/" + row)
    json = r.json()
    data[row] = {
        "country": json["country"],
        "countryCode": json["countryCode"],
        "region": json["region"],
        "regionName": json["regionName"],
        "city": json["city"],
        "zip": json["zip"],
        "lat": json["lat"],
        "lon": json["lon"],
        "org": json["org"],
        "as": json["as"],
        "ip": row
    }
    time.sleep(1.5)
df_all_doq = pd.DataFrame.from_dict(data, orient = "index")
df_all_doq.to_csv("adoption-ip-location.csv", index = False)

In [ ]:
df_all_doq

## By Continent

In [ ]:
countriesContinents = pd.read_csv("Countries-Continents.csv", index_col = False)
renamed = countriesContinents.columns.tolist()
renamed[0] = "country"
countriesContinents.columns = renamed

def mapToContinent(row):
    if row["region"] == "Americas" and row["intermediate-region"] == "South America":
        return "South America (SA)"
    elif row["region"] == "Americas":
        return "North America (NA)"
    elif row["region"] == "Asia":
        return "Asia (AS)"
    elif row["region"] == "Europe":
        return "Europe (EU)"
    elif row["region"] == "Oceania":
        return "Oceania (OC)"
    elif row["region"] == "Africa":
        return "Africa (AF)"
    return row["region"]

countriesContinents["continent"] = countriesContinents.apply(mapToContinent, axis = 1)

In [ ]:
df_all_doq_with_continent = pd.merge(df_all_doq, countriesContinents, on = "country")
df_all_doq_with_continent['continent'].value_counts()

In [ ]:
target_dist_by_continent = pd.concat([df_all_doq_with_continent['continent'].value_counts(),
                                     df_all_doq_with_continent['continent'].value_counts(normalize=True)], axis=1)
target_dist_by_continent = target_dist_by_continent.reset_index() 
target_dist_by_continent.columns = ['Continent', 'abs', 'rel']
target_dist_by_continent['rel'] = target_dist_by_continent['rel'].map(" ({:.2%})".format)
target_dist_by_continent['DoQ verified'] = target_dist_by_continent['abs'].astype(str) + target_dist_by_continent['rel']
target_dist_by_continent = target_dist_by_continent[['Continent', 'DoQ verified']]
print(target_dist_by_continent.to_latex(index=False))

In [ ]:
# 0 resolvers are missing continent information
len(df_all_doq) - df_all_doq_with_continent['continent'].value_counts().sum()

## By ASN

In [ ]:
def lookupASName(asn):
    r = requests.get("http://api.asrank.caida.org/v2/restful/asns/" + str(asn))
    json = r.json()
    return json["data"]["asn"]["asnName"]

top_n = 10

target_dist_by_asn = pd.concat([df_all_doq_with_continent['as'].value_counts().head(top_n),
                                     df_all_doq_with_continent['as'].value_counts(normalize=True).head(top_n)], axis=1)
target_dist_by_asn = target_dist_by_asn.reset_index() 
target_dist_by_asn.columns = ['ASN', 'abs', 'rel']
target_dist_by_asn['rel'] = target_dist_by_asn['rel'].map(" ({:.2%})".format)
target_dist_by_asn_table = target_dist_by_asn
target_dist_by_asn_table['DoQ verified'] = target_dist_by_asn['abs'].astype(str) + target_dist_by_asn['rel']

target_dist_by_asn_table['ASN'] = target_dist_by_asn_table['ASN'].str.split(' ', expand=True)[0]
target_dist_by_asn_table['ASN'] = target_dist_by_asn_table['ASN'].str.slice(2)

target_dist_by_asn_table['AS'] = target_dist_by_asn_table.apply(lambda row : lookupASName(row['ASN']), axis=1)

target_dist_asn_list = target_dist_by_asn_table[['ASN']]


In [ ]:
target_dist_by_asn_table['ASN'] = target_dist_by_asn_table['AS'] + ' (AS' + target_dist_by_asn_table['ASN'].astype(str) + ')'

target_dist_by_asn_table = target_dist_by_asn_table[['ASN', 'DoQ verified']]

print(target_dist_by_asn_table.to_latex(index=False))

In [ ]:
# calculate "other"
print((len(df_all_doq) - target_dist_by_asn['abs'].sum()) / len(df_all_doq))
print(len(df_all_doq) - target_dist_by_asn['abs'].sum())

## Continental distribution of top 10 ASN

In [ ]:
as_df_all_doq_with_continent = df_all_doq_with_continent[['as', 'continent']]
as_df_all_doq_with_continent['as_number'] = as_df_all_doq_with_continent['as'].str.split(' ', expand=True)[0]
as_df_all_doq_with_continent['as_number'] = as_df_all_doq_with_continent['as_number'].str.slice(2)

top_10_as_df_all_doq_with_continent = as_df_all_doq_with_continent[as_df_all_doq_with_continent['as_number'].isin(target_dist_asn_list['ASN'])]
top_10_as_df_all_doq_with_continent = pd.concat([top_10_as_df_all_doq_with_continent['continent'].value_counts(),
                                     top_10_as_df_all_doq_with_continent['continent'].value_counts(normalize=True)], axis=1)
top_10_as_df_all_doq_with_continent = top_10_as_df_all_doq_with_continent.reset_index() 
top_10_as_df_all_doq_with_continent.columns = ['Continent', 'abs', 'rel']
top_10_as_df_all_doq_with_continent['rel'] = top_10_as_df_all_doq_with_continent['rel'].map(" {:.2%}".format)
top_10_as_df_all_doq_with_continent


## Continental and ASN distribution of AdGuard and NextDNS

In [ ]:
ip_df_all_doq_with_continent = df_all_doq_with_continent[['as', 'continent', 'ip', 'country']]
ip_df_all_doq_with_continent['as_number'] = ip_df_all_doq_with_continent['as'].str.split(' ', expand=True)[0]
ip_df_all_doq_with_continent['as_number'] = ip_df_all_doq_with_continent['as_number'].str.slice(2)
ip_df_all_doq_with_continent

### AdGuard

In [ ]:
quic_doq_versions_adguard_last_week = quic_doq_versions_adguard[quic_doq_versions_adguard['WeekNumber'] == '2022-03']
adguard_last_week_df_all_doq_with_continent = ip_df_all_doq_with_continent[ip_df_all_doq_with_continent['ip'].isin(quic_doq_versions_adguard_last_week['ip'])]
adguard_last_week_df_all_doq_with_continent['asn'] = adguard_last_week_df_all_doq_with_continent.apply(lambda row : lookupASName(row['as_number']), axis=1)

In [ ]:
adguard_last_week_df_all_doq_with_continent_grouped = adguard_last_week_df_all_doq_with_continent.groupby(['as', 'asn', 'continent', 'country']).size().reset_index(name = 'count')
adguard_last_week_df_all_doq_with_continent_grouped


### NextDNS

In [ ]:
quic_doq_versions_nextdns_last_week = quic_doq_versions_nextdns[quic_doq_versions_nextdns['WeekNumber'] == '2022-03']
nextdns_last_week_df_all_doq_with_continent = ip_df_all_doq_with_continent[ip_df_all_doq_with_continent['ip'].isin(quic_doq_versions_nextdns_last_week['ip'])]

In [ ]:
nextdns_last_week_df_all_doq_with_continent_grouped = nextdns_last_week_df_all_doq_with_continent.groupby(['as', 'continent', 'country']).size().reset_index(name = 'count')
nextdns_last_week_df_all_doq_with_continent_grouped

In [ ]:
# 0 resolver is missing continent information
count_by_common_name[count_by_common_name.index == 'dns.nextdns.io']['count'] - nextdns_last_week_df_all_doq_with_continent['continent'].value_counts().sum()

In [ ]:
nextdns_last_week_df_all_doq_with_continent['as'].value_counts()

nextdns_last_week_df_all_doq_with_continent_as = pd.concat([nextdns_last_week_df_all_doq_with_continent['as'].value_counts(),
                                     nextdns_last_week_df_all_doq_with_continent['as'].value_counts(normalize=True)], axis=1)
nextdns_last_week_df_all_doq_with_continent_as = nextdns_last_week_df_all_doq_with_continent_as.reset_index() 
nextdns_last_week_df_all_doq_with_continent_as.columns = ['as', 'abs', 'rel']
nextdns_last_week_df_all_doq_with_continent_as['rel'] = nextdns_last_week_df_all_doq_with_continent_as['rel'].map(" {:.2%}".format)
nextdns_last_week_df_all_doq_with_continent_as


In [ ]:
nextdns_last_week_df_all_doq_with_continent_continent = pd.concat([nextdns_last_week_df_all_doq_with_continent['continent'].value_counts(),
                                     nextdns_last_week_df_all_doq_with_continent['continent'].value_counts(normalize=True)], axis=1)
nextdns_last_week_df_all_doq_with_continent_continent = nextdns_last_week_df_all_doq_with_continent_continent.reset_index() 
nextdns_last_week_df_all_doq_with_continent_continent.columns = ['Continent', 'abs', 'rel']
nextdns_last_week_df_all_doq_with_continent_continent['rel'] = nextdns_last_week_df_all_doq_with_continent_continent['rel'].map(" {:.2%}".format)
nextdns_last_week_df_all_doq_with_continent_continent

In [ ]:
nextdns_last_week_df_all_doq_with_continent['country'].value_counts()

In [ ]:
df_all_doq_with_continent

In [ ]:
plt.figure(figsize=(12,4))
worldMap = Basemap(lon_0=0, resolution='l')
worldMap.drawcountries(color='#ffffff', linewidth=0.5)
worldMap.fillcontinents(color='#c0c0c0', lake_color='#ffffff')
x, y = worldMap(df_all_doq_with_continent["lon"].tolist(), df_all_doq_with_continent["lat"].tolist())
homeX, homeY = worldMap([11.66864063068705], [48.26187023547006])
plt.plot(x, y, 'ro', markersize = 3)
plt.plot(homeX, homeY, 'bo', markersize = 4)

conts_joined = 'Asia: 550 (45.19%)\nEU: 394 (32.37%)\nNA: 217 (17.83%)\nOC: 30 (2.47%)\nSA: 18 (1.48%)\nAF: 8 (0.66%)'
plt.annotate(conts_joined, xy=(99.8, 40), xycoords='axes points',
            size=10, ha='right', va='bottom',
            bbox=dict(fc='w', lw='0.8'))
plt.box(False)
plt.show()


# ---------------------------------------------------------------------------------

# Check verified resolvers against public resolver ip list

In [ ]:
public_resolver_ipv4s = pd.read_csv('public-resolvers-ipv4s.csv', names = ['ip'])

In [ ]:
# no match

df_all_doq[df_all_doq['ip'].isin(public_resolver_ipv4s['ip'])]

# ---------------------------------------------------------------------------------

# Weekly port distribution

In [ ]:
sqlQuicPorts = """
select 
    strftime('%Y-%W', created_at) WeekNumber,
    draft_version as doq_version, ip, port
from q_versions_cleaned;
"""

quic_ports = pd.read_sql_query(sqlQuicPorts, connection_q_versions_cleaned)
quic_ports['port'] = quic_ports['port'].apply(str)
quic_ports["doq_ports"] = quic_ports["doq_version"] + "/" + quic_ports["port"]

quic_ports_grouped = quic_ports.groupby(['WeekNumber', 'doq_ports'])['ip'].size().reset_index(name = "total_per_week")
quic_ports_grouped
quic_ports_grouped_pivot = quic_ports_grouped.pivot(index = "WeekNumber", columns = "doq_ports", values = "total_per_week")
quic_ports_grouped_pivot = quic_ports_grouped_pivot.astype(pd.Int64Dtype())
quic_ports_grouped_pivot = quic_ports_grouped_pivot.fillna(0)
quic_ports_grouped_pivot

quic_ports_grouped_pivot

In [ ]:
quic_ports_grouped_pivot['784'] = quic_ports_grouped_pivot['doq-i00/784'] + quic_ports_grouped_pivot['doq-i02/784'] + quic_ports_grouped_pivot['doq-i03/784']

In [ ]:
quic_ports_grouped_pivot = quic_ports_grouped_pivot[['doq-i02/853', 'doq-i02/8853', '784']]
quic_ports_grouped_pivot['853'] = quic_ports_grouped_pivot['doq-i02/853'];
quic_ports_grouped_pivot['8853'] = quic_ports_grouped_pivot['doq-i02/8853']

quic_ports_grouped_pivot = quic_ports_grouped_pivot.drop(['doq-i02/853', 'doq-i02/8853'],axis=1)

quic_ports_grouped_pivot['sum'] = quic_ports_grouped_pivot.sum(axis=1)
quic_ports_grouped_pivot['853 rel'] = (quic_ports_grouped_pivot['853'] / quic_ports_grouped_pivot['sum']).map(" {:.2%}".format)
quic_ports_grouped_pivot['8853 rel'] = (quic_ports_grouped_pivot['8853'] / quic_ports_grouped_pivot['sum']).map(" {:.2%}".format)
quic_ports_grouped_pivot['784 rel'] = (quic_ports_grouped_pivot['784'] / quic_ports_grouped_pivot['sum']).map(" {:.2%}".format)

quic_ports_grouped_pivot


# IPs with doq on multiple ports

In [ ]:
sqlQuicUncleanedPorts = """
select 
    strftime('%Y-%W', created_at) WeekNumber,
    draft_version as doq_version, ip, port
from q_versions;
"""

quic_uncleaned_ports = pd.read_sql_query(sqlQuicUncleanedPorts, connection_complete)
quic_uncleaned_ports['port'] = quic_uncleaned_ports['port'].apply(str)
quic_uncleaned_ports["doq_ports"] = quic_ports["doq_version"] + "/" + quic_ports["port"]

quic_ports_grouped_ips = quic_uncleaned_ports.groupby(['WeekNumber', 'ip']).agg(lambda col: ','.join(col))
quic_ports_grouped_ips = quic_ports_grouped_ips[(quic_ports_grouped_ips['doq_version'] != 'doq-i00') & (quic_ports_grouped_ips['doq_version'] != 'doq-i02') & (quic_ports_grouped_ips['doq_version'] != 'doq-i03')]
quic_ports_grouped_ips = quic_ports_grouped_ips.reset_index()


In [ ]:
print(len(quic_ports_grouped_ips))
print(quic_ports_grouped_ips['port'].unique())
#471 ip/week combionations with multiple ports, where the only combination is 784,8853


In [ ]:
quic_ports_grouped_ips[quic_ports_grouped_ips['ip'].isin(adguard_ips)]
len(quic_ports_grouped_ips)
# all multiple port usage are attributed to adguad 


# ---------------------------------------------------------------------------------

# Increase in usage of DoQ-02 / QUIC 1 in weeks 50-01

In [ ]:
quic_certs_complete_dropped_duplicates = quic_certs_last.drop_duplicates(subset = ['ip'])
quic_certs_complete_dropped_duplicates

In [ ]:
quic_doq_versions_w50 = quic_doq_versions[quic_doq_versions['WeekNumber'] == '2021-50'][['ip', 'doq_quic']]
quic_doq_versions_w51 = quic_doq_versions[quic_doq_versions['WeekNumber'] == '2021-51'][['ip', 'doq_quic']]
quic_doq_versions_w52 = quic_doq_versions[quic_doq_versions['WeekNumber'] == '2021-52'][['ip', 'doq_quic']]
quic_doq_versions_w01 = quic_doq_versions[quic_doq_versions['WeekNumber'] == '2022-01'][['ip', 'doq_quic']]

quic_doq_versions_doq02_quic1_rampup_w51 = pd.merge(quic_doq_versions_w50, quic_doq_versions_w51, left_on='ip', right_on='ip', how='left')
quic_doq_versions_doq02_quic1_rampup_w51.columns=['ip', 'doq_quic_50', 'doq_quic_51']
quic_doq_versions_doq02_quic1_rampup_w51 = quic_doq_versions_doq02_quic1_rampup_w51[(quic_doq_versions_doq02_quic1_rampup_w51['doq_quic_50'] != 'DoQ_Draft_02/QUIC_1') & (quic_doq_versions_doq02_quic1_rampup_w51['doq_quic_51'] == 'DoQ_Draft_02/QUIC_1')]
quic_doq_versions_doq02_quic1_rampup_w51 = pd.merge(quic_doq_versions_doq02_quic1_rampup_w51, quic_certs_complete_dropped_duplicates[['ip', 'common_name']], left_on='ip', right_on='ip', how='left')

quic_doq_versions_doq02_quic1_rampup_w52 = pd.merge(quic_doq_versions_w51, quic_doq_versions_w52, left_on='ip', right_on='ip', how='left')
quic_doq_versions_doq02_quic1_rampup_w52.columns=['ip', 'doq_quic_51', 'doq_quic_52']
quic_doq_versions_doq02_quic1_rampup_w52 = quic_doq_versions_doq02_quic1_rampup_w52[(quic_doq_versions_doq02_quic1_rampup_w52['doq_quic_51'] != 'DoQ_Draft_02/QUIC_1') & (quic_doq_versions_doq02_quic1_rampup_w52['doq_quic_52'] == 'DoQ_Draft_02/QUIC_1')]
quic_doq_versions_doq02_quic1_rampup_w52 = pd.merge(quic_doq_versions_doq02_quic1_rampup_w52, quic_certs_complete_dropped_duplicates[['ip', 'common_name']], left_on='ip', right_on='ip', how='left')

quic_doq_versions_doq02_quic1_rampup_w01 = pd.merge(quic_doq_versions_w52, quic_doq_versions_w01, left_on='ip', right_on='ip', how='left')
quic_doq_versions_doq02_quic1_rampup_w01.columns=['ip', 'doq_quic_52', 'doq_quic_01']
quic_doq_versions_doq02_quic1_rampup_w01 = quic_doq_versions_doq02_quic1_rampup_w01[(quic_doq_versions_doq02_quic1_rampup_w01['doq_quic_52'] != 'DoQ_Draft_02/QUIC_1') & (quic_doq_versions_doq02_quic1_rampup_w01['doq_quic_01'] == 'DoQ_Draft_02/QUIC_1')]
quic_doq_versions_doq02_quic1_rampup_w01 = pd.merge(quic_doq_versions_doq02_quic1_rampup_w01, quic_certs_complete_dropped_duplicates[['ip', 'common_name']], left_on='ip', right_on='ip', how='left')


In [ ]:
#increase from w50 to w51-w01 are mostly from DoQ_Draft_02/QUIC_Draft_34
print(quic_doq_versions_doq02_quic1_rampup_w51['doq_quic_50'].unique())
print(quic_doq_versions_doq02_quic1_rampup_w52['doq_quic_51'].unique())
print(quic_doq_versions_doq02_quic1_rampup_w01['doq_quic_52'].unique())

#we find 'adguard' multiple times in the common names
print(len(quic_doq_versions_doq02_quic1_rampup_w51[quic_doq_versions_doq02_quic1_rampup_w51['common_name'].str.contains('adguard').fillna(False)]))
print(len(quic_doq_versions_doq02_quic1_rampup_w52[quic_doq_versions_doq02_quic1_rampup_w52['common_name'].str.contains('adguard').fillna(False)]))
print(len(quic_doq_versions_doq02_quic1_rampup_w01[quic_doq_versions_doq02_quic1_rampup_w01['common_name'].str.contains('adguard').fillna(False)]))

#checking the aguard home implementation, we find that AdGuard Home v0.107.0 adds QUIC 1 support, and was released on 21.12.2021 which is week 51 (see https://github.com/AdguardTeam/AdGuardHome/releases/tag/v0.107.0)
